In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
# import plotly.io as pio
# pio.renderers.default='notebook'
from  sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing  import LabelEncoder
from sklearn import linear_model 
from sklearn import tree 
from sklearn import ensemble 
from sklearn import metrics 
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest, f_classif

In [4]:
df = pd.read_csv('data/bank_fin.csv', sep = ';')

### Задание 3

In [30]:
df.balance.value_counts(ascending=False)

0  $            774
 1,00 $          39
 3,00 $          34
 2,00 $          34
 4,00 $          29
               ... 
 1 641,00 $       1
 5 517,00 $       1
 8 332,00 $       1
 4 859,00 $       1
-134,00 $         1
Name: balance, Length: 3802, dtype: int64

In [28]:
a = np.nan
a is not np.nan

False

In [5]:
def to_float(x):
    if x is not np.nan:
        x = x.replace(',00', '')
        x = x.replace(' $', '')
        x = x.replace(' ', '')
        return float(x)
    else:
        return x

In [6]:
df.balance = df.balance.apply(to_float)

In [6]:
round(df.balance.mean(), 3)

1529.129

### Задание 2.4

In [7]:
df.balance.fillna(df.balance.median(), inplace=True)
df.balance.isnull().sum()

0

In [8]:
round(df.balance.mean(), 3)

1526.936

### Задание 2.5

In [65]:
df.job.mode()

0    management
Name: job, dtype: object

In [66]:
df.education.mode()

0    secondary
Name: education, dtype: object

In [8]:
def to_mode(df, features):
    for feature in features:
        df[feature] = df[feature].replace('unknown', df[feature].mode()[0])
    return df

In [9]:
df = to_mode(df=df,features= ['job', 'education'])

In [11]:
df.education.value_counts()

secondary    5973
tertiary     3689
primary      1500
Name: education, dtype: int64

In [72]:
df.education.mode()[0]

'secondary'

In [12]:

mask_job = df.job == df.job.mode()[0]
mask_edu = df.education == df.education.mode()[0]
round(df[mask_job & mask_edu].balance.mean(), 3)

1598.883

### Задание 2..6

In [8]:
def outliers_z_score_mod(data, feature, log_scale=False, left=3, right=3):
    
    """
    Finds outliers in data using the z-deviation method.
    The classical method is modified by adding:
      * possibility of logarithmic distribution
      * manual control of the number of standard deviations in both sides of the distribution.
      
    Args:
        feature (str): the name of the attribute based on which outliers are searched
        left (float, optional):number of standard deviations to the left side of the distribution, default 3
        right (float, optional): number of standard deviations to the right side of the distribution, default 3
        log_scale(bool, optional): logarithm mode, default False
        
    Returns:
        pandas.DataFrame: outlier observations
        pandas.DataFrame: cleaned data from which outliers are excluded.
    """
    
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
        
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

In [20]:
df.balance.describe()

count    11162.000000
mean      1526.936033
std       3224.829691
min      -6847.000000
25%        123.000000
50%        550.000000
75%       1699.000000
max      81204.000000
Name: balance, dtype: float64

In [10]:
def outliers_iqr(data, feature, left=1.5, right=1.5):
    
    """
    Finds outliers in data using the Tukey method.
    The classical method is modified by adding:
      * possibility of logarithmic distribution
      * manual control of the number of standard deviations in both sides of the distribution.
      
    Args:
        feature (str): the name of the attribute based on which outliers are searched
        left (float, optional): number of interquartile range to the left side of box border, default 1.5
        right (float, optional): number of interquartile range to the right side of box border, default 1.5
        
        
    Returns:
        pandas.DataFrame: outlier observations
        pandas.DataFrame: cleaned data from which outliers are excluded
        float: left bound of outliers
        float: right bound of outliers
    """
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>=lower_bound) & (x <= upper_bound)]
    return outliers, cleaned, lower_bound, upper_bound 

In [11]:
outliers, df_cleaned, lower, upper = outliers_iqr(df, 'balance')

In [57]:
print(round(lower))
print(round(upper))

-2241
4063


In [16]:
fig = px.histogram(
    df,
    x='balance',
    nbins=180,
    title='Распределение признака balance',
    height=600,
    marginal='box'
)
fig.show()
fig.write_html("visualization/hist_balance.html")

### Задание 3.1

In [15]:
fig = px.histogram(
    data_frame=df_cleaned, 
    x='deposit',
    title='Диаграмма сбалансированности классов'
    )
fig.show()